<a href="https://colab.research.google.com/github/saralstalin/FAISS/blob/main/VectorProductSearchWithFAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -qU langchain-community langchain_openai faiss-gpu gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/1

In [2]:
import faiss, csv
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS as FaissVectorStore
from google.colab import userdata
from uuid import uuid4
from langchain_core.documents import Document

In [3]:
#need private API Key to use Open AI embeddings
openAIAPIKey = userdata.get('OpenAIAPIKey')
OpenAITextembeddings = OpenAIEmbeddings(model="text-embedding-3-small"
                                        , api_key=openAIAPIKey)

In [4]:
index = faiss.IndexFlatL2(len(OpenAITextembeddings.embed_query("")))

#define vector store
vector_store = FaissVectorStore(
    embedding_function=OpenAITextembeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [5]:
#read from products.csv, ignore conversion issues
with open('/content/Products.csv', 'r') as f:
    reader = csv.reader(f)
    documents = []
    for row in reader:
        try:
          price = float(row[2])
          product = Document(page_content=row[0]
                             , metadata={"Quantity": row[1]
                                         , "Price": price})
          documents.append(product)
        except ValueError:
          print(f"Error converting price: {row[2]}")

vector_store.add_documents(documents)



Error converting price: Price


['a2efef86-76a0-41cc-9f41-0bbc4322c658',
 'a237893b-527d-408f-9372-841b7a2cfebe',
 '3cc20434-2041-462d-a830-895137a7344b',
 'b165c325-1732-48ba-a4e5-0efbbe8bffce',
 '4a7f5170-9334-4ebe-9fd9-4465670885c5',
 '598a7a41-953f-4925-ad47-cc8d125a47ac',
 'e0748296-d731-4e39-802a-c957748ac62d',
 '419bacea-89f3-46a4-b1a3-e49b83f6c774',
 '0a7fb09c-0686-4ed9-9544-bdf439893abb',
 'c46b1f94-f362-4452-b90f-134489e85e81',
 '88d5aba4-900f-4476-8541-2a7f0c1d8fb6',
 '40ad0d21-74a1-45c8-85a6-20a7526e2622',
 '114e21f5-dc6a-4783-870a-956672d264f7',
 '62ad95a5-a99e-4274-b6dd-0f1c06774037',
 '094feed6-c21d-4219-b6bd-70ee774131f5',
 'ade68510-b83f-4431-a98a-6e7864078de3',
 'ebf89078-0aa0-4547-bc0f-45bc2604ba13',
 '1aef2d1f-f663-47ea-93e9-95340c773ff0',
 'de9fd6fe-13db-4510-bd45-bcd3a534d796',
 '9d08302b-89d2-4e77-9b27-4e2f01140b8e',
 '9c00452f-dbe5-4531-81f1-252978c91736',
 '9290ea5b-63ff-41a4-af05-0e3088e30100',
 'cc500c62-bc82-405d-88c2-c890115cd103',
 'fd907eb0-31cc-4e91-96c6-57eb7ce1f819',
 'e19c3985-3927-

In [72]:
def FindProducts(SearchTerm
                 , PriceMin = 0
                 , PriceMax = 999999
                 , NumberOfResults = 10
                 , MaxSimilarityScore = 2):

    results = vector_store.similarity_search_with_score(SearchTerm
                                                        , k=NumberOfResults,)

    filteredResults = [res for res, score in results
                        if res.metadata["Price"] >= PriceMin
                           and res.metadata["Price"] <= PriceMax
                           and score <= MaxSimilarityScore]

    if len(filteredResults) == 0:
        return "No results found"

    #return filtered_results

    # # Sort the results by similarity score
    #filteredResults.sort(key=lambda x: x[1])

    response = "|Product|Quantity|Price|\n"
    response += "|---|---|---|\n"

    #generate a result array with page_content and price alone
    for res in filteredResults:
       response += "|" + res.page_content + "|" + res.metadata["Quantity"] + "|" + str(res.metadata["Price"] ) + "|\n"

    return response



In [73]:
searchResult = FindProducts("apple", PriceMax=200, MaxSimilarityScore=2)

print(searchResult)

|Product|Quantity|Price|
|---|---|---|
|Apples|1kg |100.0|
|Beans|1kg |50.0|
|Pineapple|1kg |50.0|
|Peaches|1kg |150.0|
|Oranges|1kg |80.0|
|Grapes|1kg |100.0|



In [18]:
import gradio as gr

In [74]:
iface = gr.Interface(
    fn=FindProducts,
    inputs=[
        gr.Textbox(label="Search..."),
        gr.Number(label="Price Min", value=0),
        gr.Number(label="Price Max", value=999999),
        gr.Number(label="Number of Results", value=10),
        gr.Number(label="Match Rate", value=2),
    ],
    outputs=gr.Markdown(),
    title="Product Search",
    description="Enter a search term and get product suggestions.",
    submit_btn="Search",
)

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://de2a9c0f9eca11fa64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
